# Importing Libraries

* System Append to set proper path

In [171]:
sys.path.append('../Source/Utils')

* Default

In [172]:
import lasio
import pandas as pd
import numpy as np
from collections import Counter

* Pandas Options

In [173]:
pd.set_option('max_columns', None)

* Source Code

In [174]:
import welllog

* Tqdm Progress Bar

In [175]:
%%capture
from tqdm import tqdm_notebook

# Reading Miscellaneous Data

In [ ]:
file_path = '../data/Lithology code data.xlsx'

file_path2 = '../data/NPD stratigraphic picks north sea.xlsx'

litho_code = pd.read_excel(file_path)

picks = pd.read_excel(file_path2)

* Lithology Code 

In [ ]:
litho_code.head

        * Fixing Columns IDs

In [ ]:
litho_code.columns = litho_code.iloc[0]

        * Droping first row (as it has become the columns ids now)

In [ ]:
litho_code.drop(index=0, inplace=True)

        * Checking Final Result

In [ ]:
litho_code.head(5)

* Stratigraphic Picks

In [ ]:
picks.head(n=2)

        * Renaming Unidentified Column

In [ ]:
picks.rename(columns={'Unnamed: 0': 'Unidentified Column'}, inplace=True)

        * Checking Result

In [ ]:
picks.head(n=2)

# Reading Las files Dataset

In [176]:
# Example for 2 wells

las = lasio.read('..//data/GEOLINK_Lithology and wells NORTH SEA/6_3-1.las')
las1 = lasio.read('..//data/GEOLINK_Lithology and wells NORTH SEA/6_3-2.las')

In [177]:
df = las.df()
df1 = las1.df()

# Creating a column with the well name

df['WELL_NAME'] = '6_3-1'
df1['WELL_NAME'] = '6_3-2'

df1

,LITHOLOGY_GEOLINK,CALI,DRHO,NPHI,RHOB,GR,DTC,RDEP,SP,RSHA,RMED,WELL_NAME
DEPT,,,,,,,,,,,,
211.500000,NaN,25.3834,NaN,NaN,NaN,22.092501,56.560101,10000.0,-26.289,NaN,0.2967,6_3-2
211.625000,NaN,25.3834,0.2529,NaN,2.5175,22.094299,56.538799,10000.0,-26.289,NaN,0.2970,6_3-2
211.749985,NaN,25.3834,0.2529,NaN,2.5175,21.809799,56.517502,10000.0,-26.289,NaN,0.2973,6_3-2
211.875000,NaN,25.3834,0.2529,NaN,2.5175,21.382900,56.496098,10000.0,-26.289,NaN,0.2976,6_3-2
212.000000,NaN,25.3834,0.2529,NaN,2.5175,21.104000,56.474800,10000.0,-26.289,NaN,0.2979,6_3-2
...,...,...,...,...,...,...,...,...,...,...,...,...
4099.375000,NaN,3.8114,NaN,NaN,NaN,NaN,NaN,NaN,67.459,NaN,NaN,6_3-2
4099.500000,NaN,3.8114,NaN,NaN,NaN,NaN,NaN,NaN,67.459,NaN,NaN,6_3-2
4099.625000,NaN,3.8114,NaN,NaN,NaN,NaN,NaN,NaN,67.459,NaN,NaN,6_3-2


In [178]:
# List of curves that need to be filtered from the wells dataframes

lista = ['LITHOLOGY_GEOLINK', 'NPHI', 'RHOB', 'GR', 'DTC', 'RDEP', 'WELL_NAME']

In [179]:
# Creating a main empty dataframe

col = {'LITHOLOGY_GEOLINK':[], 'CALI':[], 'NPHI':[], 'RHOB':[], 'GR':[], 'DTC':[], 'RDEP':[], 'WELL_NAME':[]}
df_main = pd.DataFrame(columns=list(col))

df_main

,LITHOLOGY_GEOLINK,CALI,NPHI,RHOB,GR,DTC,RDEP,WELL_NAME


In [180]:
# Appending the dataframes of two wells

df_main = df_main.append(df[lista])
df_main = df_main.append(df1[lista])


df_main

# ^^^^ Aqui a ideia é fazer um laço onde os indices serão todos os arquivos las do diretório '..//data/GEOLINK_Lithology and wells NORTH SEA/'

,LITHOLOGY_GEOLINK,CALI,NPHI,RHOB,GR,DTC,RDEP,WELL_NAME
107.125000,NaN,NaN,NaN,1.0,NaN,205.999985,0.000000,6_3-1
107.250000,NaN,NaN,NaN,1.0,8.580900,205.999985,4810.740234,6_3-1
107.375000,NaN,NaN,NaN,1.0,12.589000,205.999985,7057.797852,6_3-1
107.500000,NaN,NaN,NaN,1.0,16.597099,205.999985,9304.855469,6_3-1
107.625008,NaN,NaN,NaN,1.0,17.837000,205.999985,10000.000000,6_3-1
...,...,...,...,...,...,...,...,...
4099.375000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6_3-2
4099.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6_3-2
4099.625000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6_3-2
4099.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6_3-2


* Checking unmatching unit of measurement for each log curve

In [ ]:
unit_mismatch_list = welllog.unit_check(npd_wells)

* Converting all las files to dataframe